In [1]:
# Celda 1: Importar librerías
import os
import pandas as pd
from sqlalchemy import create_engine, text
# --- Importar tipos de SQLAlchemy ---
from sqlalchemy import Integer, String, Text, TIMESTAMP, Boolean, VARCHAR # Añadido VARCHAR
from dotenv import load_dotenv
import logging
import numpy as np
import warnings

In [2]:
# Celda 2: Configuración
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
pd.set_option('display.max_rows', 15)
pd.set_option('display.max_columns', None)
# Suprimir la advertencia específica de zona horaria mixta si es necesario, aunque es mejor solucionarla.
# warnings.filterwarnings("ignore", message="parsing datetimes with mixed time zones will raise an error", category=FutureWarning)

Celda 3: Establecer conexión a la base de datos y cargar datos en DataFrame


In [3]:
# Celda 3: Cargar Variables de Entorno y Conectar (Asume que el engine se creó antes)
logging.info("Cargando variables de entorno...")
dotenv_path = '/home/nicolas/Escritorio/workshops/workshop_2/env/.env' # <-- CONFIRMA ESTA RUTA
load_dotenv(dotenv_path=dotenv_path)

DB_USER = os.getenv('POSTGRES_USER')
DB_PASSWORD = os.getenv('POSTGRES_PASSWORD')
DB_HOST = os.getenv('POSTGRES_HOST')
DB_PORT = os.getenv('POSTGRES_PORT')
DB_NAME = os.getenv('POSTGRES_DB')
SOURCE_TABLE_NAME = 'the_grammy_awards' # Tabla original de donde leer
CLEAN_TABLE_NAME = 'the_grammy_awards_clean' # Nombre de la nueva tabla limpia

df = None
engine = None

if not all([DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME]):
    logging.error("Faltan variables de entorno para la base de datos en " + dotenv_path)
    raise ValueError("Variables de entorno incompletas.")
else:
    try:
        db_url = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
        engine = create_engine(db_url)
        logging.info(f"Motor SQLAlchemy creado para la base de datos '{DB_NAME}'.")
    except Exception as e:
        logging.error(f"Error al crear el motor SQLAlchemy: {e}")
        raise

2025-04-10 23:59:07,487 - INFO - Cargando variables de entorno...
2025-04-10 23:59:07,515 - INFO - Motor SQLAlchemy creado para la base de datos 'artists'.


In [4]:
# Celda 4: Cargar Datos Originales desde PostgreSQL
if engine:
    try:
        logging.info(f"Leyendo datos desde la tabla original '{SOURCE_TABLE_NAME}'...")
        query = f"SELECT * FROM \"{SOURCE_TABLE_NAME}\"" # Usar comillas por si acaso
        df = pd.read_sql_query(query, con=engine)
        logging.info(f"Datos originales cargados exitosamente desde '{SOURCE_TABLE_NAME}'. Filas: {len(df)}")
    except Exception as e:
        logging.error(f"Error al leer la tabla original '{SOURCE_TABLE_NAME}': {e}")
        df = None # Asegurarse de que df sea None si falla la carga
        raise
else:
    logging.error("No se pudo crear el engine. No se pueden cargar los datos.")
    raise ValueError("Engine no disponible.")

# --- Inicio de la Limpieza ---


2025-04-10 23:59:07,524 - INFO - Leyendo datos desde la tabla original 'the_grammy_awards'...
2025-04-10 23:59:07,561 - INFO - Datos originales cargados exitosamente desde 'the_grammy_awards'. Filas: 4810


In [5]:
# Celda 5: Crear copia para la limpieza
if df is not None:
    logging.info(f"Creando copia del DataFrame para limpieza: '{CLEAN_TABLE_NAME}'")
    the_grammy_awards_clean = df.copy()
else:
    logging.error("No se puede iniciar la limpieza porque 'df' no está cargado.")
    raise ValueError("DataFrame 'df' original no disponible para copiar.")

2025-04-10 23:59:07,571 - INFO - Creando copia del DataFrame para limpieza: 'the_grammy_awards_clean'


In [6]:
# Celda 6: Conversión de Tipos de Datos (Fechas) - CORREGIDO
if the_grammy_awards_clean is not None:
    logging.info("Convirtiendo columnas de fecha a datetime (UTC)...")
    date_cols = ['published_at', 'updated_at']
    initial_non_nulls = the_grammy_awards_clean[date_cols].notnull().sum()

    for col in date_cols:
        if col in the_grammy_awards_clean.columns:
            try:
                # --- CORRECCIÓN: Añadir utc=True ---
                the_grammy_awards_clean[col] = pd.to_datetime(the_grammy_awards_clean[col], errors='coerce', utc=True)
                # ------------------------------------
                logging.info(f"Columna '{col}' convertida a datetime (UTC). Nuevo tipo: {the_grammy_awards_clean[col].dtype}")
            except Exception as e:
                 logging.error(f"Error convirtiendo '{col}' a datetime: {e}")
                 # Mantenerla como objeto si falla inesperadamente
                 pass # Opcional: the_grammy_awards_clean[col] = df[col] para restaurar

    # Verificar errores de conversión (NaT)
    final_non_nulls = the_grammy_awards_clean[date_cols].notnull().sum()
    for col in date_cols:
         if col in final_non_nulls.index: # Asegurar que la columna aún exista
             errors_count = initial_non_nulls.get(col, 0) - final_non_nulls.get(col, 0)
             if errors_count > 0:
                 logging.warning(f"Se encontraron {errors_count} errores de formato en '{col}' que se convirtieron a NaT.")

2025-04-10 23:59:07,582 - INFO - Convirtiendo columnas de fecha a datetime (UTC)...
2025-04-10 23:59:07,594 - INFO - Columna 'published_at' convertida a datetime (UTC). Nuevo tipo: datetime64[ns, UTC]
2025-04-10 23:59:07,599 - INFO - Columna 'updated_at' convertida a datetime (UTC). Nuevo tipo: datetime64[ns, UTC]


In [7]:
# Celda 7: Manejo de Valores Nulos
if the_grammy_awards_clean is not None:
    logging.info("Manejando valores nulos...")

    # 1. Eliminar filas con 'nominee' nulo
    initial_rows = len(the_grammy_awards_clean)
    the_grammy_awards_clean.dropna(subset=['nominee'], inplace=True)
    rows_dropped = initial_rows - len(the_grammy_awards_clean)
    if rows_dropped > 0:
        logging.info(f"Se eliminaron {rows_dropped} filas por nulos en 'nominee'.")
    else:
         logging.info("No se encontraron nulos en 'nominee'.")

    # 2. Rellenar nulos en otras columnas clave
    fill_values = {
        'artist': 'No Especificado',
        'workers': 'No Especificado',
        'img': 'Sin URL'
    }
    logging.info(f"Rellenando nulos en {list(fill_values.keys())}...")
    for col, placeholder in fill_values.items():
        if col in the_grammy_awards_clean.columns:
             null_count_before = the_grammy_awards_clean[col].isnull().sum()
             if null_count_before > 0:
                 the_grammy_awards_clean[col].fillna(placeholder, inplace=True)
                 logging.info(f"  - Nulos en '{col}' rellenados con '{placeholder}'. ({null_count_before} valores)")
             else:
                 logging.info(f"  - No se encontraron nulos en '{col}'.")

    logging.info("Verificación de nulos después del relleno:")
    print(the_grammy_awards_clean[['artist', 'workers', 'img']].isnull().sum())

2025-04-10 23:59:07,609 - INFO - Manejando valores nulos...
2025-04-10 23:59:07,614 - INFO - Se eliminaron 6 filas por nulos en 'nominee'.
2025-04-10 23:59:07,615 - INFO - Rellenando nulos en ['artist', 'workers', 'img']...
2025-04-10 23:59:07,616 - INFO -   - Nulos en 'artist' rellenados con 'No Especificado'. (1834 valores)
2025-04-10 23:59:07,619 - INFO -   - Nulos en 'workers' rellenados con 'No Especificado'. (2184 valores)
2025-04-10 23:59:07,622 - INFO -   - Nulos en 'img' rellenados con 'Sin URL'. (1361 valores)
2025-04-10 23:59:07,623 - INFO - Verificación de nulos después del relleno:


artist     0
workers    0
img        0
dtype: int64


In [8]:
# Celda 8: Limpieza Básica de Texto (Espacios)
if the_grammy_awards_clean is not None:
    logging.info("Eliminando espacios en blanco iniciales/finales de columnas de texto...")
    # Seleccionar solo columnas que son *definitivamente* de texto después de otras conversiones
    object_columns_final = the_grammy_awards_clean.select_dtypes(include=['object']).columns
    cols_stripped = []
    for col in object_columns_final:
        if the_grammy_awards_clean[col].notnull().any(): # Solo si hay algo que limpiar
            try:
                the_grammy_awards_clean[col] = the_grammy_awards_clean[col].str.strip()
                cols_stripped.append(col)
            except AttributeError:
                 logging.warning(f"No se pudo aplicar .str.strip() a la columna '{col}'.") # Raro si es tipo object
    if cols_stripped:
        logging.info(f"Espacios eliminados en columnas: {cols_stripped}")
    else:
        logging.info("No se encontraron columnas de texto con espacios para limpiar.")

2025-04-10 23:59:07,649 - INFO - Eliminando espacios en blanco iniciales/finales de columnas de texto...
2025-04-10 23:59:07,673 - INFO - Espacios eliminados en columnas: ['title', 'category', 'nominee', 'artist', 'workers', 'img']


In [9]:
# Celda 9: Conversión a Tipos de Pandas Óptimos (Opcional pero recomendado antes de to_sql)
if the_grammy_awards_clean is not None:
    logging.info("Optimizando tipos de datos en Pandas...")
    try:
        # Convertir columnas de texto a 'string' de pandas (más eficiente que 'object')
        for col in ['title', 'category', 'nominee', 'artist', 'workers', 'img']:
             if col in the_grammy_awards_clean.columns:
                 the_grammy_awards_clean[col] = the_grammy_awards_clean[col].astype('string')
        logging.info("Columnas de texto convertidas a tipo 'string' de Pandas.")
    except Exception as e:
        logging.warning(f"No se pudieron convertir todas las columnas a 'string': {e}")

2025-04-10 23:59:07,687 - INFO - Optimizando tipos de datos en Pandas...
2025-04-10 23:59:07,696 - INFO - Columnas de texto convertidas a tipo 'string' de Pandas.


In [10]:
# Celda 10: Verificación Final del DataFrame Limpio
if the_grammy_awards_clean is not None:
    logging.info("\n--- Verificación del DataFrame Limpio ('the_grammy_awards_clean') ---")
    print("\nPrimeras filas del DataFrame limpio:")
    display(the_grammy_awards_clean.head())
    print("\nInformación del DataFrame limpio (tipos en Pandas):")
    # Mostrar info para ver los tipos de Pandas actualizados
    the_grammy_awards_clean.info()
    print("\nConteo de nulos por columna en el DataFrame limpio:")
    display(the_grammy_awards_clean.isnull().sum())

2025-04-10 23:59:07,707 - INFO - 
--- Verificación del DataFrame Limpio ('the_grammy_awards_clean') ---



Primeras filas del DataFrame limpio:


,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
0,2019,62nd Annual GRAMMY Awards (2019),2020-05-19 12:10:28+00:00,2020-05-19 12:10:28+00:00,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",https://www.grammy.com/sites/com/files/styles/...,True
1,2019,62nd Annual GRAMMY Awards (2019),2020-05-19 12:10:28+00:00,2020-05-19 12:10:28+00:00,Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",https://www.grammy.com/sites/com/files/styles/...,True
2,2019,62nd Annual GRAMMY Awards (2019),2020-05-19 12:10:28+00:00,2020-05-19 12:10:28+00:00,Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",https://www.grammy.com/sites/com/files/styles/...,True
3,2019,62nd Annual GRAMMY Awards (2019),2020-05-19 12:10:28+00:00,2020-05-19 12:10:28+00:00,Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",https://www.grammy.com/sites/com/files/styles/...,True
4,2019,62nd Annual GRAMMY Awards (2019),2020-05-19 12:10:28+00:00,2020-05-19 12:10:28+00:00,Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",https://www.grammy.com/sites/com/files/styles/...,True



Información del DataFrame limpio (tipos en Pandas):
<class 'pandas.core.frame.DataFrame'>
Index: 4804 entries, 0 to 4809
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   year          4804 non-null   int64              
 1   title         4804 non-null   string             
 2   published_at  4804 non-null   datetime64[ns, UTC]
 3   updated_at    4804 non-null   datetime64[ns, UTC]
 4   category      4804 non-null   string             
 5   nominee       4804 non-null   string             
 6   artist        4804 non-null   string             
 7   workers       4804 non-null   string             
 8   img           4804 non-null   string             
 9   winner        4804 non-null   bool               
dtypes: bool(1), datetime64[ns, UTC](2), int64(1), string(6)
memory usage: 380.0 KB

Conteo de nulos por columna en el DataFrame limpio:


year            0
title           0
published_at    0
updated_at      0
category        0
nominee         0
artist          0
workers         0
img             0
winner          0
dtype: int64

In [11]:
# Celda 11: Carga del DataFrame Limpio a PostgreSQL con TIPOS SQL EXPLÍCITOS
if the_grammy_awards_clean is not None and engine is not None:

    # --- Definir los tipos de datos SQL deseados ---
    # Basado en tu solicitud y buenas prácticas
    sql_types = {
        'year': Integer(),                      # int64
        'title': Text(),                        # string (Pandas) -> TEXT (SQL)
        'published_at': TIMESTAMP(timezone=True),# datetime64[ns, UTC] -> TIMESTAMP WITH TIME ZONE
        'updated_at': TIMESTAMP(timezone=True),  # datetime64[ns, UTC] -> TIMESTAMP WITH TIME ZONE
        'category': Text(),                     # string (Pandas) -> TEXT
        'nominee': Text(),                      # string (Pandas) -> TEXT
        'artist': Text(),                       # string (Pandas) -> TEXT
        'workers': Text(),                      # string (Pandas) -> TEXT (VARCHAR podría truncar)
        'img': Text(),                          # string (Pandas) -> TEXT (VARCHAR(2083) o más si prefieres)
        'winner': Boolean()                     # bool -> BOOLEAN
    }
    # -------------------------------------------

    logging.info(f"Cargando DataFrame limpio a la tabla '{CLEAN_TABLE_NAME}' en la base de datos '{DB_NAME}' con tipos explícitos...")
    try:
        the_grammy_awards_clean.to_sql(
            CLEAN_TABLE_NAME,
            con=engine,
            if_exists='replace',
            index=False,
            method='multi',
            dtype=sql_types
        )
        logging.info(f"DataFrame limpio cargado exitosamente en la tabla '{CLEAN_TABLE_NAME}'.")

        # Verificación final en la base de datos
        logging.info(f"Verificando número de filas en la tabla '{CLEAN_TABLE_NAME}'...")
        with engine.connect() as connection:
            query_count = text(f"SELECT COUNT(*) FROM \"{CLEAN_TABLE_NAME}\"")
            result = connection.execute(query_count)
            num_db_clean_rows = result.scalar_one()

        logging.info(f"Número de filas en la tabla limpia '{CLEAN_TABLE_NAME}': {num_db_clean_rows}")
        logging.info(f"Número de filas en el DataFrame limpio: {len(the_grammy_awards_clean)}")

        if len(the_grammy_awards_clean) == num_db_clean_rows:
            logging.info("¡Verificación de carga exitosa!")
        else:
            logging.warning("Discrepancia en el número de filas entre el DataFrame limpio y la tabla cargada.")

    except Exception as e:
        logging.error(f"Error al cargar el DataFrame limpio en la base de datos: {e}")

elif the_grammy_awards_clean is None:
     logging.error("No se puede cargar la tabla limpia porque el DataFrame 'the_grammy_awards_clean' no está definido.")
elif engine is None:
     logging.error("No se puede cargar la tabla limpia porque la conexión a la base de datos (engine) no está definida.")


logging.info("--- Proceso de Limpieza y Carga Finalizado ---")

2025-04-10 23:59:07,751 - INFO - Cargando DataFrame limpio a la tabla 'the_grammy_awards_clean' en la base de datos 'artists' con tipos explícitos...
2025-04-10 23:59:08,603 - INFO - DataFrame limpio cargado exitosamente en la tabla 'the_grammy_awards_clean'.
2025-04-10 23:59:08,604 - INFO - Verificando número de filas en la tabla 'the_grammy_awards_clean'...
2025-04-10 23:59:08,607 - INFO - Número de filas en la tabla limpia 'the_grammy_awards_clean': 4804
2025-04-10 23:59:08,607 - INFO - Número de filas en el DataFrame limpio: 4804
2025-04-10 23:59:08,607 - INFO - ¡Verificación de carga exitosa!
2025-04-10 23:59:08,608 - INFO - --- Proceso de Limpieza y Carga Finalizado ---
